In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from matplotlib import pyplot as plt

import medmnist
from medmnist import INFO, Evaluator

In [2]:
lap_egns_1 = torch.load("eigenvalues/lap_egns_1_up_pneum.pt")
lap_egns_2 = torch.load("eigenvalues/lap_egns_2_up_pneum.pt")
lap_egns_3 = torch.load("eigenvalues/lap_egns_3_up_pneum.pt")
lap_egns_4 = torch.load("eigenvalues/lap_egns_4_up_pneum.pt")
lap_egns_5 = torch.load("eigenvalues/lap_egns_5_up_pneum.pt")

lap_egns_test = torch.load("eigenvalues/lap_egns_test_up_pneum.pt")

train_labels = torch.load("eigenvalues/train_labels_pneum.pt")
test_labels = torch.load("eigenvalues/test_labels_pneum.pt")

In [3]:
train_eigns = torch.cat([lap_egns_1, lap_egns_2, lap_egns_3, lap_egns_4, lap_egns_5])

In [4]:
train_eigns.shape

torch.Size([4708, 14])

In [5]:
train_data = [(lap, lb) for lap, lb in zip(train_eigns, train_labels.squeeze().tolist())]  # train_eigns[:, :, 1] 

In [6]:
test_data = [(lap, lb) for lap, lb in zip(lap_egns_test, test_labels.squeeze().tolist())] # lap_egns_test[:, :, 1] 

# Training

In [7]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64,
                                           shuffle=True)

In [8]:
train_eigns.shape

torch.Size([4708, 14])

In [9]:
model = nn.Sequential(
            nn.Linear(14, 2048),
            nn.Tanh(),
            nn.Linear(2048, 2048),
            nn.Tanh(),
            nn.Linear(2048, 1024),
            nn.Tanh(),
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Linear(512, 256),
            nn.Tanh(),
            nn.Linear(256, 2),
            # nn.LogSoftmax(dim=1)
)

In [10]:
learning_rate = 5e-3

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss()

n_epochs = 150

In [11]:
for epoch in range(n_epochs):
    for eig_vals, label in train_loader:
        out = model(eig_vals)
        loss = loss_fn(out, label)
                
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))
    # if loss <= 0.1:
    #     break

Epoch: 0, Loss: 0.631846
Epoch: 1, Loss: 0.630686
Epoch: 2, Loss: 0.650990
Epoch: 3, Loss: 0.439146
Epoch: 4, Loss: 0.629814
Epoch: 5, Loss: 0.673261
Epoch: 6, Loss: 0.477947
Epoch: 7, Loss: 0.517186
Epoch: 8, Loss: 0.568032
Epoch: 9, Loss: 0.582032
Epoch: 10, Loss: 0.630030
Epoch: 11, Loss: 0.449050
Epoch: 12, Loss: 0.533164
Epoch: 13, Loss: 0.437833
Epoch: 14, Loss: 0.615019
Epoch: 15, Loss: 0.573433
Epoch: 16, Loss: 0.630515
Epoch: 17, Loss: 0.495899
Epoch: 18, Loss: 0.542592
Epoch: 19, Loss: 0.669660
Epoch: 20, Loss: 0.633032
Epoch: 21, Loss: 0.579963
Epoch: 22, Loss: 0.490449
Epoch: 23, Loss: 0.477770
Epoch: 24, Loss: 0.439067
Epoch: 25, Loss: 0.598364
Epoch: 26, Loss: 0.634819
Epoch: 27, Loss: 0.543206
Epoch: 28, Loss: 0.644784
Epoch: 29, Loss: 0.472721
Epoch: 30, Loss: 0.521462
Epoch: 31, Loss: 0.440084
Epoch: 32, Loss: 0.482989
Epoch: 33, Loss: 0.504768
Epoch: 34, Loss: 0.449371
Epoch: 35, Loss: 0.540571
Epoch: 36, Loss: 0.451022
Epoch: 37, Loss: 0.480611
Epoch: 38, Loss: 0.407

In [12]:
# train_loader = torch.utils.data.DataLoader(train_data, batch_size=64,
#                                            shuffle=False)
correct = 0
total = 0

with torch.no_grad():
    for laps, labels in train_loader:
        outputs = model(laps)
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Accuracy: %f" % (correct / total))

Accuracy: 0.905480


In [13]:
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64,
                                         shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for laps, labels in test_loader:
        outputs = model(laps)
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Accuracy: %f" % (correct / total))

Accuracy: 0.857372
